<a href="https://colab.research.google.com/github/theaidran/ollama_youtube_summarize/blob/main/ollama_youtube_summarize.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# @title #####  Put a YouTube link below and hit Run a cell. Use any-language video and get any-language summary. Quick access to this colab notebook. [tinyurl.com/summarisation](https://tinyurl.com/summarisation) { display-mode: "form" }
%config InteractiveShell.ast_node_interactivity="all" #update python display config

In [3]:
# @title  Summarize { run: "auto", display-mode: "form" }
# All credits to https://github.com/martinopiaggi/summarize
# Good to see links:
# https://github.com/hsiehjackson/RULER
# https://ollama.com/library
# https://amgadhasan.substack.com/p/sota-asr-tooling-long-form-transcription


#1 Choose source url
Link = "https://www.youtube.com/watch?v=t6VoqbAh9HM" #@param {type:"string"}
Type_of_source = "Youtube video or playlist" #@param ['Youtube video or playlist', 'Google Drive video link','Dropbox video link', 'Local file']
Type = Type_of_source
URL = Link
Translate_to = "English" # @param ['Afrikaans', 'Albanian', 'Amharic', 'Arabic', 'Armenian', 'Azerbaijani', 'Basque', 'Belarusian', 'Bengali', 'Bosnian', 'Bulgarian', 'Catalan', 'Cebuano', 'Chinese (Simplified)', 'Chinese (Traditional)', 'Corsican', 'Croatian', 'Czech', 'Danish', 'Dutch', 'English', 'Esperanto', 'Estonian', 'Finnish', 'French', 'Frisian', 'Galician', 'Georgian', 'German', 'Greek', 'Gujarati', 'Haitian Creole', 'Hausa', 'Hawaiian', 'Hebrew', 'Hindi', 'Hmong', 'Hungarian', 'Icelandic', 'Igbo', 'Indonesian', 'Irish', 'Italian', 'Japanese', 'Javanese', 'Kannada', 'Kazakh', 'Khmer', 'Korean', 'Kurdish', 'Kyrgyz', 'Lao', 'Latin', 'Latvian', 'Lithuanian', 'Luxembourgish', 'Macedonian', 'Malagasy', 'Malay', 'Malayalam', 'Maltese', 'Maori', 'Marathi', 'Mongolian', 'Myanmar (Burmese)', 'Nepali', 'Norwegian', 'Nyanja (Chichewa)', 'Pashto', 'Persian', 'Polish', 'Portuguese (Portugal, Brazil)', 'Punjabi', 'Romanian', 'Russian', 'Samoan', 'Scots Gaelic', 'Serbian', 'Sesotho', 'Shona', 'Sindhi', 'Sinhala (Sinhalese)', 'Slovak', 'Slovenian', 'Somali', 'Spanish', 'Sundanese', 'Swahili', 'Swedish', 'Tagalog (Filipino)', 'Tajik', 'Tamil', 'Telugu', 'Thai', 'Turkish', 'Ukrainian', 'Urdu', 'Uzbek', 'Vietnamese', 'Welsh', 'Xhosa', 'Yiddish', 'Yoruba', 'Zulu']

language_codes = {'Afrikaans': 'af', 'Albanian': 'sq', 'Amharic': 'am', 'Arabic': 'ar', 'Armenian': 'hy', 'Azerbaijani': 'az', 'Basque': 'eu', 'Belarusian': 'be', 'Bengali': 'bn', 'Bosnian': 'bs', 'Bulgarian': 'bg', 'Catalan': 'ca', 'Cebuano': 'ceb', 'Chinese (Simplified)': 'zh-CN', 'Chinese (Traditional)': 'zh-TW', 'Corsican': 'co', 'Croatian': 'hr', 'Czech': 'cs', 'Danish': 'da', 'Dutch': 'nl', 'English': 'en', 'Esperanto': 'eo', 'Estonian': 'et', 'Finnish': 'fi', 'French': 'fr', 'Frisian': 'fy', 'Galician': 'gl', 'Georgian': 'ka', 'German': 'de', 'Greek': 'el', 'Gujarati': 'gu', 'Haitian Creole': 'ht', 'Hausa': 'ha', 'Hawaiian': 'haw', 'Hebrew': 'iw', 'Hindi': 'hi', 'Hmong': 'hmn', 'Hungarian': 'hu', 'Icelandic': 'is', 'Igbo': 'ig', 'Indonesian': 'id', 'Irish': 'ga', 'Italian': 'it', 'Japanese': 'ja', 'Javanese': 'jw', 'Kannada': 'kn', 'Kazakh': 'kk', 'Khmer': 'km', 'Korean': 'ko', 'Kurdish': 'ku', 'Kyrgyz': 'ky', 'Lao': 'lo', 'Latin': 'la', 'Latvian': 'lv', 'Lithuanian': 'lt', 'Luxembourgish': 'lb', 'Macedonian': 'mk', 'Malagasy': 'mg', 'Malay': 'ms', 'Malayalam': 'ml', 'Maltese': 'mt', 'Maori': 'mi', 'Marathi': 'mr', 'Mongolian': 'mn', 'Myanmar (Burmese)': 'my', 'Nepali': 'ne', 'Norwegian': 'no', 'Nyanja (Chichewa)': 'ny', 'Pashto': 'ps', 'Persian': 'fa', 'Polish': 'pl', 'Portuguese (Portugal, Brazil)': 'pt', 'Punjabi': 'pa', 'Romanian': 'ro', 'Russian': 'ru', 'Samoan': 'sm', 'Scots Gaelic': 'gd', 'Serbian': 'sr', 'Sesotho': 'st', 'Shona': 'sn', 'Sindhi': 'sd', 'Sinhala (Sinhalese)': 'si', 'Slovak': 'sk', 'Slovenian': 'sl', 'Somali': 'so', 'Spanish': 'es', 'Sundanese': 'su', 'Swahili': 'sw', 'Swedish': 'sv', 'Tagalog (Filipino)': 'tl', 'Tajik': 'tg', 'Tamil': 'ta', 'Telugu': 'te', 'Thai': 'th', 'Turkish': 'tr', 'Ukrainian': 'uk', 'Urdu': 'ur', 'Uzbek': 'uz', 'Vietnamese': 'vi', 'Welsh': 'cy', 'Xhosa': 'xh', 'Yiddish': 'yi', 'Yoruba': 'yo', 'Zulu': 'zu'}

#@markdown ---
#markdown Insert your API key depending on which endpoint you want to use
# API Configuration
api_key = "ollama"
api_endpoint = "Custom"
ollama_model = "llama3" # param ['llama3', 'phi3', 'vicuna:13b-v1.5-16k-q4_0']

endpoints = {
    "Groq": "https://api.groq.com/openai/v1",
    "OpenAI": "https://api.openai.com/v1",
    "Custom": "http://localhost:11434/v1"  # ollama endpoint
}
base_url = endpoints.get(api_endpoint)

models = {
    "Groq": "llama3-8b-8192",
    "OpenAI": "gpt-3.5-turbo",
    "Custom": ollama_model  # Placeholder for any custlom model
}
model = models.get(api_endpoint)

use_Youtube_captions = True #@param {type:"boolean"}
# @markdown  * otherwise transcribe audio


#2 launch ollama server with llama3
try: # to faster next start
  import pytubefix
except ImportError:
  !curl -fsSL https://ollama.com/install.sh | sh
  !tmux new -d ollama serve
  !ollama run {ollama_model} !


#3 install libs: youtube and whisperx
import subprocess
import re
import os
import IPython
from IPython.display import clear_output

def install_whisperx():
    try:
      import whisperx
    except ImportError:
      print("installing whisperx...may take a few minutes")
      !pip install git+https://github.com/m-bain/whisperx.git > /dev/null
      clear_output()

if use_Youtube_captions:
  !pip install youtube-transcript-api
  from youtube_transcript_api import YouTubeTranscriptApi

if (not Type == "Youtube video or playlist") or (not use_Youtube_captions):
  install_whisperx()

try:
  import deep_translator
except ImportError:
  !pip install -U deep-translator

try:
  import openai
except ImportError:
  !pip install openai
  #clear_output()

import openai
client = openai.OpenAI(api_key=api_key, base_url=base_url)

if Type == "Youtube video or playlist":
  try:
    import pytubefix
  except ImportError:
    #!pip install git+https://github.com/pytube/pytube
    !pip install git+https://github.com/JuanBindez/pytubefix
    #clear_output()
    from pytubefix import YouTube

if Type == "Google Drive video link":
  from google.colab import drive
  drive.mount('/gdrive')

#4 Video fetching
skip_transcription=False
text = ""
textTimestamps = ""

def seconds_to_time_format(seconds):
    hours, remainder = divmod(seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return f"{int(hours):02d}:{int(minutes):02d}:{int(seconds):02d}"

def download_youtube_audio_only(url):
    yt = YouTube(url)
    audio_stream = yt.streams.get_audio_only()
    saved_path = audio_stream.download(output_path=".", skip_existing=True)
    return saved_path

def check_if_captions_exist(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)

    try:
      transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
      return (True)
    except:
      return (False)

def download_youtube_captions(url):
    regex = r'(?:https?:\/\/)?(?:www\.)?(?:youtube\.com\/(?:[^\/\n\s]+\/\S+\/|(?:v|e(?:mbed)?)\/|\S*?[?&]v=)|youtu\.be\/)([a-zA-Z0-9_-]{11})'
    video_id =  re.search(regex, url).group(1)
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)

    try:
      transcript = YouTubeTranscriptApi.get_transcript(video_id) #languages=['en']
    except:
      for available_transcript in transcript_list:
        if available_transcript.is_translatable:
          transcript = available_transcript.translate('en').fetch()
          break

    text = ""
    for entry in transcript:
            start_time = seconds_to_time_format(entry['start'])
            text += f"{start_time} {entry['text'].strip()}\n"

    transcript_file_name = f"{video_id}_captions.md"

    with open(transcript_file_name, 'w', encoding='utf-8') as f:
      f.write(text)

    return text,transcript_file_name

if Type == "Youtube video or playlist":
    #clean youtube url from timestamp
    URL = re.sub('\&t=\d+s?', '', URL)

    if use_Youtube_captions and check_if_captions_exist(URL):
      text, transcript_file_name = download_youtube_captions(URL)
      skip_transcription=True
    else:
      install_whisperx()
      video_path_local =  download_youtube_audio_only(URL)

elif Type == "Google Drive video link":
  subprocess.run(['ffmpeg', '-y', '-i', '/gdrive/My Drive/' + URL, '-vn', '-acodec', 'pcm_s16le',
                  '-ar', '16000', '-ac', '1', 'gdrive_audio.wav'], check=True)
  video_path_local = "gdrive_audio.wav"

elif Type == "Dropbox video link":
    subprocess.run(['wget', URL, '-O', 'dropbox_video.mp4'], check=True)
    subprocess.run(['ffmpeg', '-y', '-i', 'dropbox_video.mp4', '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'dropbox_video_audio.wav'], check=True)
    video_path_local = "dropbox_video_audio.wav"

elif Type == "Local file":
    local_file_path = Source
    subprocess.run(['ffmpeg', '-y', '-i', local_file_path, '-vn', '-acodec', 'pcm_s16le',
                    '-ar', '16000', '-ac', '1', 'local_file_audio.wav'], check=True)
    video_path_local = "local_file_audio.wav"


#5 transcription using whisperx
if not skip_transcription:
  import whisperx
  language = "auto" # param {type:"string"}
  #initial_prompt = "" # @param {type:"string"}

  asr_options={"temperatures": 0, "beam_size": 1, "without_timestamps": True,}
  # Run on GPU with FP16
  modelWhisperX = whisperx.load_model("large-v2", device="cuda", compute_type="float16", asr_options=asr_options)

  transcription = modelWhisperX.transcribe(str(video_path_local), batch_size=16,
                                    language=None if language == "auto" else language,
                                    task="translate")

  transcript_file_name = os.path.splitext(video_path_local)[0]+'.md'

  with open(transcript_file_name, 'w') as f:
    for segment in transcription["segments"]:
        start_time = seconds_to_time_format(segment['start'])
        text += f"{start_time} {segment['text'].strip()} "

    f.write(text)


#6 Make a summary
prompt_type = "Summarization"  # param ['Summarization', 'Only grammar correction with highlights']
# markdown Set the number of parallel API calls (be mindful of usage rate limits)
parallel_api_calls = 1 # param


# Define your prompts using a dictionary for easier management
prompts = {
    'Summarization': """Summarize the video transcript excerpt, including a bold title. Use narration in third-person. Write the summary as if you are continuing a conversation without needing to signal a beginning. Here is the transcript: """,
    'Only grammar correction with highlights': """Repeat the following text correcting any grammatical errors and formatting error. Highlight only the important quote (if there are any) with **markdown bold notation**. Focus solely on the essence of the content as if you are continuing a conversation without using any form of introduction like 'Here's the corrected text:'. Here is the text to fix: """
}

# Select the appropriate prompt
summary_prompt = prompts[prompt_type]



def extract_and_clean_timestamps(text_chunks):
    timestamp_pattern = re.compile(r'(\d{2}:\d{2}:\d{2})')
    cleaned_texts = []
    timestamp_ranges = []
    for chunk in text_chunks:
        timestamps = timestamp_pattern.findall(chunk)
        if timestamps:
            for timestamp in timestamps:
                # Remove each found timestamp from the chunk
                chunk = chunk.replace(timestamp, "")
            timestamp_ranges.append(timestamps[0])  # Assuming you want the first timestamp per chunk
        else:
            timestamp_ranges.append("")
        cleaned_texts.append(chunk.strip())  # Strip to remove any leading/trailing whitespace
    return cleaned_texts, timestamp_ranges

def format_timestamp_link(timestamp):
    if Type == "Youtube video or playlist":
      hours, minutes, seconds = map(int, timestamp.split(':'))
      total_seconds = hours * 3600 + minutes * 60 + seconds
      return f"{timestamp} - {URL}&t={total_seconds}"
    else:
      return f"{timestamp}"

import concurrent.futures
import time
from deep_translator import GoogleTranslator

def summarize(prompt):
    completion = client.chat.completions.create(
            model=model,
            messages=[
            {"role": "system", "content": ""},
            {"role": "user", "content": summary_prompt + " " + prompt}
            ],
            max_tokens=4096
    )
    return completion.choices[0].message.content

def process_and_summarize(text,chunk_size):
    overlap_size = 20
    texts = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size - overlap_size)]
    cleaned_texts, timestamp_ranges = extract_and_clean_timestamps(texts)
    summaries = []

    with concurrent.futures.ThreadPoolExecutor(max_workers=parallel_api_calls) as executor:
        future_to_chunk = {executor.submit(summarize, text_chunk): idx for idx, text_chunk in enumerate(cleaned_texts)}
        for future in concurrent.futures.as_completed(future_to_chunk):
            idx = future_to_chunk[future]
            try:
                summarized_chunk = future.result()
                summary_piece = format_timestamp_link(timestamp_ranges[idx]) + " " + summarized_chunk
                summary_piece += "\n"
                summaries.append((idx, summary_piece))
            except Exception as exc:
                print(f'Chunk {idx} generated an exception: {exc}')
                # Resubmit the task with the new model
                time.sleep(10)
                future_to_chunk[executor.submit(summarize, texts[idx])] = idx

    summaries.sort()  # Ensure summaries are in the correct order
    final_summary =""
    if Translate_to != "English":
      translated_summaries = []
      for _,summary in summaries:
        translated_summary = GoogleTranslator(source='auto', target=language_codes[Translate_to]).translate(summary)
        translated_summaries.append(translated_summary)
        final_summary += translated_summary + "\n\n"
      final_summary = final_summary.strip()
    else :
      final_summary = "\n\n".join([summary for _, summary in summaries])

    # Save the final summary
    final_name = transcript_file_name.replace(".md", "_FINAL_"+str(chunk_size)+".md") if Type != "Dropbox video link" else "final_dropbox_video.md"
    with open(final_name, 'w') as f:
        f.write(final_summary)
    return (final_summary)

from IPython.core.interactiveshell import InteractiveShell # Unblock output
InteractiveShell.ast_node_interactivity = "all"            # It starts to work after restart cell
#%config InteractiveShell.ast_node_interactivity="all"
clear_output()
print ("Generating summary...")
general_summary = process_and_summarize(text,15000) # max string size = max_tokens x 4
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary)
final_summary = process_and_summarize(text,3072)
clear_output()
IPython.display.Markdown("\n\n **Short Summary:** \n\n" + general_summary + "\n\n" + "\n\n **Long Summary:** \n\n" + final_summary )





 **Short Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **Generating Content with AI: A Step-by-Step Guide**

In this video transcript excerpt, the speaker discusses using artificial intelligence (AI) tools like ChatGPT and Claude to generate content for their website. They start by writing a prompt that provides details about their site, which reviews drones, and specify what they need - website copies that will entice visitors to use the site.

To generate the content, the speaker uses Claude and creates a header and subheader in the style of Alex Horoi and David Ogili. The AI tool produces a heading "The Ultimate Drone Buyer Guide: Soaring to New Heights with the Perfect UAV" and a subheading "Discover the Best Drones for Your Needs and Budget: Expert Reviews, Side-by-Side Comparisons, and Insider Tips to Help You Make an Informed Decision".

The speaker then uses this generated content as the heading and subheading on their website. They also explore using AI tools like Mid Journey and Dolly 3 to generate images for their website, including a popular camera drone flying above a city skyline.

Finally, they discuss creating featured images for blog posts using Canva, which is easy to use and allows them to add text, remove backgrounds, and manipulate image elements. Overall, the speaker provides a step-by-step guide on how to utilize AI tools to generate content for their website, making it more engaging and informative for visitors.


00:11:15 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=675 **Creating a Featured Image Using Canva's AI Features**

As the narrator continues to showcase the power of AI tools, he delves into creating a featured image using Canva. He starts by clicking on "Create a design" and selecting the option to create a YouTube thumbnail, which is also suitable for blog featured images. The narrator then uses Canva's Magic Media feature to generate an image based on his preferences, in this case, a camera drone flying above a city skyline. He right-clicks on the generated image and sets it as the background, adding some personal touch by uploading an image of himself pointing at the drone.

To make himself stand out, he uses AI to remove the background from the uploaded image and adds a white outline around him. He then selects the font "Roboto Condensed" for the text overlay, which is consistent with his website's style. Finally, he saves the featured image as a JPEG and downloads it, ready to be used on his website.

This segment highlights how AI can streamline the process of creating high-quality featured images for blog posts, making it easier to maintain consistency across the site.




 **Long Summary:** 

00:00:00 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=0 **"AI-Enabled Website Building Tools: Getting Started with Reloom"**

The conversation continues, discussing AI-enabled tools for building websites without requiring technical knowledge. The speaker emphasizes the importance of having a website and highlights Reloom as a valuable tool for mapping out an entire site using artificial intelligence. With Reloom, users can create a simple website on the free plan or upgrade to premium plans that integrate with web design tools like Webflow and Figma.

The speaker walks viewers through creating a new project in Reloom, starting by entering a prompt about their website's content. In this example, they're building a site reviewing and comparing camera drones. After clicking "Generate Site Map," the AI starts to build out a basic layout for the homepage and subsequent pages, including hero sections, feature lists, reviews, pricing information, testimonials, FAQs, and footers.

The speaker then demonstrates how to generate content for each page using Reloom's AI capabilities. They can click on individual Pages to create design concepts and even scroll through wireframes to get a rough idea of what the site will look like.


00:02:18 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=138 **Building a Website with Hostinger's AI Tool**

As we continue, we're now focusing on building a website using Hostinger's AI-powered tool. After selecting the Business website builder plan, we can choose to let AI create our website in minutes. We enter the coupon code "Matt wolf" and click apply to receive an even bigger discount.

Once inside the hosting or website builder, we select the option to let AI create our website. We input the brand name "Drone World", choose a blog format for the description, and speak out the description for our website - a platform that compares and reviews camera drones to help consumers make informed decisions.

With just a few clicks, AI sets everything up for us, and we have a live website already titled "Drone World" with automatically pulled-in pictures of drone photography. Now, let's edit the site and change anything we want, starting with the color scheme using ColorMind, a tool that utilizes machine learning technology...


00:04:34 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=274 **Elevating Website Design with AI: Color Palette Generation**

The speaker is using Colormind, an AI-powered tool, to generate a color palette for their website. They start by generating a random palette and then lock in the colors they like, which are a blue, yellow, and red. By locking these colors in, the AI generates more variations that match well with the selected design. The speaker is pleased with the result and uses the generated codes to apply the color scheme to their website.

**Font Selection: A Perfect Match**

Next, the speaker turns their attention to font selection using Fontjoy, another AI-powered tool. They sit through a few different font styles until they find one that stands out - Robotto Condensed. They lock this font in and then use the AI to generate other fonts that pair well with it. After a couple of clicks, they're satisfied with their findings and select the new font for their website's headers and subheadings.


00:06:48 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=408 **Crafting Website Content with AI**

The speaker is continuing to work on building their website, focusing on creating the content section. They're using AI tools like ChatGPT or Claude to generate the text, as they've already established a font pairing and color scheme. The goal is to create engaging copy that will encourage visitors to use the site. To achieve this, the speaker starts by giving details about their website, which reviews and compares drones to help consumers make informed decisions. They then provide specific instructions for the AI tool, including writing a header and subheader in the style of Alex Horoi and David Ogie.


00:09:01 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=541 **Website Optimization Strategies**

As our conversation continues, we're discussing the importance of optimizing website headings and subheadings. We've learned that it's crucial to repeat this step for each section of your website, making sure to copy and replace the heading and subheading as needed.

The speaker also highlights the need for high-quality images throughout the website. While they appreciate the image used in this example, they want to generate more images using tools like Mid Journey, Dolly 3, or Canva. These tools allow users to create realistic images with a personal touch, making them more effective for enhancing website content.

We've also explored how to edit and customize these generated images using settings like model raw and adjusting the image size. The speaker shares their experience in generating and editing an image of a drone flying over a city skyline, replacing the original image on the website.

In addition to optimizing headings and subheadings, we've touched on the importance of creating relevant featured images for blog posts. Canva is presented as a favorite tool for this task, allowing users to add text overlays, remove backgrounds, and edit images with ease.


00:11:27 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=687 **Creating Consistent Featured Images with Canva**

As the speaker navigates through Canva, they're looking for ways to create consistent featured images for their blog posts. They start by exploring the Magic media option and using the built-in image generator to create custom graphics. After generating a few options, they choose one that resembles a drone flying above a city skyline, which will serve as the background for their featured image.

Next, they select an image of themselves pointing, but it has a background that needs to be removed. They use Canva's AI-powered background remover tool (available only in the paid version) to remove the unwanted background and create a clean image of themselves. To make themselves stand out more, they add a white outline around their figure.

Finally, the speaker adds text to the image, selecting the "Roboto Condensed" font style to match their website's branding. They stretch out the text to cover the entire image, creating a visually appealing featured image for their blog post.


00:13:31 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=811 **Building a Website with AI: The Ultimate Guide**

As the creator of the website continues, they highlight the importance of adding visual effects to make their featured image "pop off the screen." They use Canva's AI features to create the image, adding a drop shadow effect to give it depth. With the image saved as a JPEG, they jump back to their website and replace the previous featured image with the new one.

The creator notes that they've used various AI tools throughout the process, including Reloom for site mapping, Hostinger (with a coupon code) for building the website itself, ColorMind for figuring out a color scheme, FontJoy for font pairings, and either ChatGPT or Claude for writing copy. They also used an image generator to generate images for the website.

The final step is creating featured images for blog posts using Canva. The creator emphasizes that while AI can't yet produce a beautiful, polished website from scratch, it's possible to get very close with the right tools and some extra tweaking.


00:15:55 - https://www.youtube.com/watch?v=t6VoqbAh9HM&t=955 **Staying Up-to-Date on AI Tools and News**

The speaker is encouraging viewers to check out Future Tools, a platform that curates cool AI tools and keeps users up-to-date on the latest AI news. The channel's YouTube page features new updates almost daily, with a free newsletter sent to subscribers with the coolest tools and most important news. As an added incentive, the speaker will be giving away free gadgets each month to subscribers of both the YouTube channel and the Future Tools newsletter.

To enter the drawing for this month's prize, an Insta 360 X4 camera, viewers must simply subscribe to the YouTube channel and join the free newsletter at Future Tools. This is not a one-time offer; every single month, the speaker will be giving away more cool gadgets to subscribers of both platforms. The goal is to provide valuable tools and inspiration to those building websites or working on creative projects.
